In [2]:
pip install torch torchvision torchaudio


     ------------------------------------ 212.4/212.4 MB 980.0 kB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 1.0 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 45.8/45.8 kB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 6.3/6.3 MB 1.1 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0
  Attempting uninstall: sympy
    Found existing installation: sympy 1.10.1
    Uninstalling sympy-1.10.1:
      Successfully uninstalled sympy-1.10.1
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# 1. CONFIGURATION
DATA_DIR      = r'D:\STUDY\Sumit\lectures sumit\college\sem 6\dyslexia\Dataset Dyslexia_Password WanAsy321\Gambo\Train'   # your Train folder
BATCH_SIZE    = 64
IMG_SIZE      = 64
NUM_CLASSES   = 3
EPOCHS        = 30
LR            = 1e-3
PATIENCE      = 5     # for early stopping
DEVICE        = 'cuda' if torch.cuda.is_available() else 'cpu'
CLASS_NAMES   = ['normal','reversal','corrected']

# 2. DATA AUGMENTATION & LOADER
train_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((IMG_SIZE,IMG_SIZE)),
    transforms.RandomRotation(8),
    transforms.RandomAffine(0, translate=(0.1,0.1), scale=(0.9,1.1), shear=5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),                    # [0,1]
    transforms.Normalize([0.5], [0.5]),       # [-1,1]
])
# We split train into train/val
full_dataset = datasets.ImageFolder(DATA_DIR, transform=train_transforms)
val_size = int(0.2 * len(full_dataset))
train_size = len(full_dataset) - val_size
train_ds, val_ds = random_split(full_dataset, [train_size, val_size])
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=4)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# 3. MODEL DEFINITION (Light CNN)
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(),
            nn.MaxPool2d(2),                # 32×32×32
            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.MaxPool2d(2),                # 64×16×16
            nn.Conv2d(64,128,3,padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.MaxPool2d(2),                # 128×8×8
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.3),
            nn.Linear(128*8*8, 256), nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )
    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

model = SimpleCNN(NUM_CLASSES).to(DEVICE)

# 4. TRAINING SETUP
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.5)

# Early stopping tracker
best_val_loss = np.inf
patience_ctr  = 0

# 5. TRAIN LOOP
for epoch in range(1, EPOCHS+1):
    # --- Train ---
    model.train()
    train_loss, train_acc, n = 0, 0, 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        logits = model(imgs)
        loss   = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        # metrics
        train_loss += loss.item() * imgs.size(0)
        preds = logits.argmax(dim=1)
        train_acc += (preds == labels).sum().item()
        n += imgs.size(0)
    train_loss /= n
    train_acc  = train_acc / n

    # --- Validate ---
    model.eval()
    val_loss, val_acc, n = 0, 0, 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            logits = model(imgs)
            loss   = criterion(logits, labels)
            val_loss += loss.item() * imgs.size(0)
            preds = logits.argmax(dim=1)
            val_acc += (preds == labels).sum().item()
            n += imgs.size(0)
    val_loss /= n
    val_acc  = val_acc / n
    scheduler.step(val_loss)

    print(f"Epoch {epoch}/{EPOCHS}  "
          f"Train loss: {train_loss:.4f}, acc: {train_acc:.3f}  "
          f"Val   loss: {val_loss:.4f}, acc: {val_acc:.3f}")

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_ctr  = 0
        torch.save(model.state_dict(), "best_model.pth")
    else:
        patience_ctr += 1
        if patience_ctr >= PATIENCE:
            print("Early stopping triggered.")
            break

# 6. EVALUATION ON VALIDATION SET
model.load_state_dict(torch.load("best_model.pth"))
y_true, y_pred = [], []
model.eval()
with torch.no_grad():
    for imgs, labels in val_loader:
        imgs = imgs.to(DEVICE)
        logits = model(imgs)
        preds  = logits.argmax(dim=1).cpu().numpy()
        y_pred.extend(preds)
        y_true.extend(labels.numpy())

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=CLASS_NAMES, digits=4))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES)
plt.xlabel("Predicted"); plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()


KeyboardInterrupt: 

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Paths
train_dir = r"D:\STUDY\Sumit\lectures sumit\college\sem 6\dyslexia\Dataset Dyslexia_Password WanAsy321\Gambo\Train"
test_dir = r"D:\STUDY\Sumit\lectures sumit\college\sem 6\dyslexia\Dataset Dyslexia_Password WanAsy321\Gambo\Test"

# Image params
img_size = (64, 64)
batch_size = 32

# Data generators with augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False
)

# CNN Model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64,64,1)),
    MaxPooling2D(),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training
history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=15
)

# Save model
model.save("reversal_detector.h5")


Found 151649 images belonging to 3 classes.
Found 56723 images belonging to 3 classes.


C:\Users\JAI SHREE KRISHNA\anaconda3\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\JAI SHREE KRISHNA\anaconda3\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
4740/4740 ━━━━━━━━━━━━━━━━━━━━ 1139s 240ms/step - accuracy: 0.8073 - loss: 0.4738 - val_accuracy: 0.8453 - val_loss: 0.4462
Epoch 2/15
4740/4740 ━━━━━━━━━━━━━━━━━━━━ 428s 90ms/step - accuracy: 0.9265 - loss: 0.1997 - val_accuracy: 0.8597 - val_loss: 0.4086
Epoch 3/15
4740/4740 ━━━━━━━━━━━━━━━━━━━━ 461s 97ms/step - accuracy: 0.9453 - loss: 0.1501 - val_accuracy: 0.8232 - val_loss: 0.5522
Epoch 4/15
4740/4740 ━━━━━━━━━━━━━━━━━━━━ 429s 90ms/step - accuracy: 0.9523 - loss: 0.1302 - val_accuracy: 0.8805 - val_loss: 0.3579
Epoch 5/15
4740/4740 ━━━━━━━━━━━━━━━━━━━━ 450s 95ms/step - accuracy: 0.9571 - loss: 0.1150 - val_accuracy: 0.8767 - val_loss: 0.3999
Epoch 6/15
4740/4740 ━━━━━━━━━━━━━━━━━━━━ 562s 119ms/step - accuracy: 0.9618 - loss: 0.1053 - val_accuracy: 0.8956 - val_loss: 0.3271
Epoch 7/15
4740/4740 ━━━━━━━━━━━━━━━━━━━━ 562s 118ms/step - accuracy: 0.9640 - loss: 0.0989 - val_accuracy: 0.8881 - val_loss: 0.3594
Epoch 8/15
4740/4740 ━━━━━━━━━━━━━━━━━━━━ 579s 122ms/step - accur

In [3]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available: 0


In [ ]:
!pip uninstall tensorflow 
!pip install tensorflow==2.10 --user

!conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1.0

import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))


     ------------------------------------ 455.9/455.9 MB 415.1 kB/s eta 0:00:00
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached keras-2.10.0-py2.py3-none-any.whl (1.7 MB)
  Using cached tensorflow_estimator-2.10.0-py2.py3-none-any.whl (438 kB)
  Using cached tensorboard-2.10.1-py3-none-any.whl (5.9 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.11.0
    Can't uninstall 'keras'. No files were found to uninstall.
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.11.0
    Can't uninstall 'tensorflow-estimator'. No files were found to uninstall.
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.11.0
    Can't uninstall 'tensorboard'. No files were found to uninstall.
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.11.0
    Uninstalling tensorflow-2.11.0:
      Successfully uninstalled tensorflow-2.11.0
  Rollin

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\JAI SHREE KRISHNA\\anaconda3\\Lib\\site-packages\\tensorflow\\compiler\\tf2tensorrt\\_pywrap_py_utils.pyd'
Consider using the `--user` option or check the permissions.



In [1]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))


[]


In [2]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.10.0


In [ ]:
pip install --upgrade tensorflow
